# 二項ロジスティック回帰

- 二項ロジスティック回帰とは、二値データを入力とするロジスティック回帰のこと

## 目的
- 説明変数によって出席率を予測する

## データ

- N : データの個数
- A : アルバイトが好きか否か
- Score : 学問への興味の強さ
- M : 三ヶ月間の履修登録した科目の総授業回数
- Y : そのうち実際に出席した回数

## パラメータ
- b_n : 線形回帰の切片・傾き

## モデル
出席確率
- $q[n] \sim logit^{-1}(b_{1} + b_{2} A[n] + b_{3} Score[n]/200)$

尤度関数
- $Y[n] \sim Binomial(M[n],q[n])$


In [1]:
%matplotlib inline

In [2]:
import pystan
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns


In [4]:
stan_code = """
data {
    int N;
    int<lower=0, upper=1> A[N];
    real<lower=0, upper=1> Score[N];
    int<lower=0> M[N];
    int<lower=0> Y[N];
}

parameters {
    real b1;
    real b2;
    real b3;
}

transformed parameters {
    real q[N];
    for (n in 1:N)
        q[n] =inv_logit(b1 +b2 *A[n] + b3*Score[n]);
}

model {
    for (n in 1:N)
        Y[n] ~ binomial(M[n],q[n]);
}

generated quantities {
    real y_pred[N];
    for (n in 1:N)
        y_pred[n] = binomial_rng(M[n],q[n]);
}

"""

In [13]:
df=pd.read_csv("data/data-attendance-2.txt")

stan_dat = {
    'N': len(df),
    'A': df["A"],
    'Y': df["Y"],
    'M': df["M"],
    'Score': df["Score"]/200,
}

fit = pystan.stan(model_code = stan_code, data = stan_dat, iter = 1000, chains = 4)

In [15]:
fit

Inference for Stan model: anon_model_afac82934a2f478856e07d3c5680b3bd.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

             mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
b1           0.07  8.8e-3   0.22  -0.37  -0.07   0.08   0.21   0.51    639    1.0
b2          -0.62  3.3e-3   0.09  -0.79  -0.68  -0.62  -0.56  -0.44    764    1.0
b3           1.94    0.01   0.35   1.28    1.7   1.92   2.16   2.63    641    1.0
q[0]         0.68  9.4e-4   0.02   0.62   0.66   0.68   0.69   0.72    689    1.0
q[1]          0.7  5.0e-4   0.02   0.67   0.69    0.7   0.71   0.73    978    1.0
q[2]         0.78  3.4e-4   0.01   0.76   0.77   0.78   0.79    0.8   1122    1.0
q[3]         0.57  7.2e-4   0.02   0.53   0.56   0.57   0.58   0.61    825    1.0
q[4]         0.73  6.1e-4   0.02   0.69   0.72   0.73   0.74   0.76    855    1.0
q[5]          0.8  3.4e-4   0.01   0.77   0.79    0.8    0.8   0.82   1080 

# 結果

出席確率
- $q[n] \sim logit^{-1}(0.07 -0.62 A[n] + 1.94 Score[n]/200) = \frac{1}{1+exp(-(0.07 -0.62 A[n] + 1.94 Score[n]/200))}$

出席数
- $Y[n] \sim Binomial(M[n],q[n])$